# Setup

In [1]:
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from duckduckgo_search import DDGS
from typing import TypedDict
import graphviz
import os
import yaml

In [2]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
llm = ChatOpenAI(model_name="gpt-4.1", temperature=0.3)

# Estado

In [3]:
class State(TypedDict):
    pergunta: str  
    conteudo: str 
    resposta: str 

# Funções de CallBack

In [4]:
def recebe_pergunta(state: State) -> State:
    print(f"Usuário perguntou: {state['pergunta']}")
    return {"pergunta": state["pergunta"]}

def precisa_pesquisar(state: State) -> State:
    pergunta = state["pergunta"].lower()
    precisa = any(p in pergunta for p in ["dados", "estatísticas", "números", "pesquisa"])
    print("Precisa pesquisar?", precisa)
    # Retorna um dicionário, com chave especial para decisão
    return {"next_step": "pesquisar" if precisa else "consultar_llm"}

def pesquisar(state: State) -> State:
    pergunta = state["pergunta"]
    print(f"Pesquisando no DuckDuckGo: {pergunta}")

    with DDGS() as ddgs:
        resultados = ddgs.text(pergunta, max_results=1)

    if resultados:
        contexto = "\n".join([r["body"] for r in resultados if "body" in r])
    else:
        contexto = "Nenhum resultado encontrado."

    return {"conteudo": contexto}    

def consultar_llm(state: State) -> State:
    prompt = ChatPromptTemplate.from_template("Responda à seguinte pergunta: {pergunta}")
    chain = prompt | llm
    resposta = chain.invoke({"pergunta": state["pergunta"]})
    print("Resposta direta do LLM.")
    return {"resposta": resposta.content}

def sintetizar(state: State) -> State:
    contexto = state.get("conteudo", "")
    pergunta = state["pergunta"]
    prompt = ChatPromptTemplate.from_template("""
    Use o seguinte contexto para responder a pergunta:
    Contexto: {contexto}
    Pergunta: {pergunta}
    Resposta:""")
    chain = prompt | llm
    resposta = chain.invoke({"contexto": contexto, "pergunta": pergunta})
    print("Resposta sintetizada com contexto.")
    return {"resposta": resposta.content}

def responder(state: State) -> State:
    print("\n Resposta Final:")
    print(state["resposta"])
    return state

# Cria Grafo

In [5]:
graph = StateGraph(State)

graph.add_node("recebe_pergunta", RunnableLambda(recebe_pergunta))
graph.add_node("decisao", RunnableLambda(precisa_pesquisar))
graph.add_node("pesquisar", RunnableLambda(pesquisar))
graph.add_node("consultar_llm", RunnableLambda(consultar_llm))
graph.add_node("sintetizar", RunnableLambda(sintetizar))
graph.add_node("responder", RunnableLambda(responder))

# Transições de Estado e Condições

In [6]:
graph.set_entry_point("recebe_pergunta")

graph.add_edge("recebe_pergunta", "decisao")
graph.add_conditional_edges(
    "decisao",
    lambda state: state["next_step"],  
    {
        "pesquisar": "pesquisar",
        "consultar_llm": "consultar_llm"
    }
)
graph.add_edge("pesquisar", "sintetizar")
graph.add_edge("consultar_llm", "responder")
graph.add_edge("sintetizar", "responder")
graph.set_finish_point("responder")

# Execução

In [8]:
executable = graph.compile()

print("\n TESTE 1:")
executable.invoke({"pergunta": "Qual é a capital da Alemanha?"})

print("\n  TESTE 2:")
executable.invoke({"pergunta": "Me mostre dados sobre economia brasileira em 2025."})


 TESTE 1:
Usuário perguntou: Qual é a capital da Alemanha?
Precisa pesquisar? False
Resposta direta do LLM.

 Resposta Final:
A capital da Alemanha é Berlim.

  TESTE 2:
Usuário perguntou: Me mostre dados sobre economia brasileira em 2025.
Precisa pesquisar? True
Pesquisando no DuckDuckGo: Me mostre dados sobre economia brasileira em 2025.
Resposta sintetizada com contexto.

 Resposta Final:
Segundo o Relatório de Política Monetária divulgado pelo Banco Central em 27 de junho de 2024, a estimativa de crescimento da economia brasileira para 2025 foi reduzida para abaixo de 2%. O Banco Central projeta que, ao invés de crescer 2,1% como anteriormente previsto, o Produto Interno Bruto (PIB) do Brasil deve ter um crescimento máximo de 1,9% em 2025. Essa revisão indica uma perspectiva mais cautelosa para o desempenho econômico do país no próximo ano.


{'pergunta': 'Me mostre dados sobre economia brasileira em 2025.',
 'conteudo': 'O Banco Central reduziu a estimativa de crescimento da economia brasileira em 2025 para abaixo de 2%. O número consta no Relatório de Política Monetária, divulgado nesta quinta-feira (27). A revisão dos dados mostra que o Banco Central avalia que ao invés de crescer 2,1% neste ano, a economia terá um crescimento máximo de 1,9%.',
 'resposta': 'Segundo o Relatório de Política Monetária divulgado pelo Banco Central em 27 de junho de 2024, a estimativa de crescimento da economia brasileira para 2025 foi reduzida para abaixo de 2%. O Banco Central projeta que, ao invés de crescer 2,1% como anteriormente previsto, o Produto Interno Bruto (PIB) do Brasil deve ter um crescimento máximo de 1,9% em 2025. Essa revisão indica uma perspectiva mais cautelosa para o desempenho econômico do país no próximo ano.'}

# Gráfico

In [9]:
dot = executable.get_graph().draw_mermaid()

print(dot)

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	recebe_pergunta(recebe_pergunta)
	decisao(decisao)
	pesquisar(pesquisar)
	consultar_llm(consultar_llm)
	sintetizar(sintetizar)
	responder(responder)
	__end__([<p>__end__</p>]):::last
	__start__ --> recebe_pergunta;
	consultar_llm --> responder;
	decisao -.-> consultar_llm;
	decisao -.-> pesquisar;
	pesquisar --> sintetizar;
	recebe_pergunta --> decisao;
	sintetizar --> responder;
	responder --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

